In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

Running on Apache Spark version 2.4.3
SparkUI available at http://p2050.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /net/archive/groups/plggneuromol/sportsmen-wgs/analysis/hail-20210423-1529-0.2.62-84fa81b9ea3d.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [4]:
files = !ls /net/archive/groups/plggneuromol/sportsmen-wgs/vcf-joint-gts/*gz

In [5]:
names = []

for i in files:
    i = i.split('/')
    i = i[7]
    i = i.split('.v')
    i = i[0]
    names.append(i)

In [ ]:
for idx, f in enumerate(files):
    hl.import_vcf(f, reference_genome='GRCh38', 
                  #filter='chr9[ \t]138334443', 
                  force_bgz = True, array_elements_required = False).write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/'+names[idx]+'.mt')

In [46]:
mts = []
for i in names:
    i = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/'+i+'.mt')
    mts.append(i)

In [10]:
rpmk = hl.import_bed('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/repeatmasker_all',
                     reference_genome='GRCh38', skip_invalid_intervals=True)

2021-02-11 09:59:39 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)



In [11]:
rpmk.write('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_all.ht')

2021-02-11 10:00:26 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-11 10:02:37 Hail: INFO: wrote table with 5520017 rows in 48 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_all.ht


In [12]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_all.ht')

for idx, mt in enumerate(mts):
    print(mt.count())
    mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)
    mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/rpkm-'+names[idx]+'.mt')
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/rpkm-'+names[idx]+'.mt')

In [4]:
table = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/gnomad.genomes.r3.0.coverage.summary.tsv.bgz', impute = True)

2021-02-11 15:23:40 Hail: INFO: Reading table to impute column types
2021-02-11 15:28:37 Hail: INFO: Finished type imputation
  Loading column 'locus' as type 'str' (imputed)
  Loading column 'mean' as type 'float64' (imputed)
  Loading column 'median' as type 'int32' (imputed)
  Loading column 'over_1' as type 'float64' (imputed)
  Loading column 'over_5' as type 'float64' (imputed)
  Loading column 'over_10' as type 'float64' (imputed)
  Loading column 'over_15' as type 'float64' (imputed)
  Loading column 'over_20' as type 'float64' (imputed)
  Loading column 'over_25' as type 'float64' (imputed)
  Loading column 'over_30' as type 'float64' (imputed)
  Loading column 'over_50' as type 'float64' (imputed)
  Loading column 'over_100' as type 'float64' (imputed)


In [5]:
mts = []
for i in names:
    i = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/rpkm-'+i+'.mt')
    mts.append(i)

In [6]:
table = table.select(table.locus, table.over_1)
table = table.annotate(locus = hl.parse_locus(table.locus, reference_genome='GRCh38'))
table = table.filter(table.over_1 >0.9)
table = table.key_by(table.locus)

In [ ]:
for idx, mt in enumerate(mts):
    print(mt.count())
    mt = mt.filter_rows(hl.is_defined(table[mt.locus]), keep = True)
    mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/cov-'+names[idx]+'.mt')
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/cov-'+names[idx]+'.mt')

(275903, 287)


2021-02-11 15:41:54 Hail: INFO: Coerced sorted dataset
2021-02-11 15:42:23 Hail: INFO: wrote matrix table with 258263 rows and 287 columns in 52 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/cov-part016.mt


(279497, 287)


In [25]:
gnomad = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/gnomad/gnomad_selected_filtered.ht')

In [47]:
for idx, mt in enumerate(mts):
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/cov-'+names[idx]+'.mt')
    mt = mt.select_rows(mt.rsid, mt.info.AC, mt.info.AF, mt.info.AN)
    mt = hl.split_multi_hts(mt)
    mt = mt.annotate_rows(gnomad_v3 = gnomad[mt.row_key])
    mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/gnmd-'+names[idx]+'.mt')
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/gnmd-'+names[idx]+'.mt')

2021-02-11 19:28:41 Hail: INFO: wrote matrix table with 905930 rows and 287 columns in 58 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/gnmd-part002.mt
2021-02-11 19:28:57 Hail: INFO: wrote matrix table with 685765 rows and 287 columns in 50 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/gnmd-part003.mt
2021-02-11 19:29:15 Hail: INFO: wrote matrix table with 751889 rows and 287 columns in 55 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/gnmd-part004.mt
2021-02-11 19:29:31 Hail: INFO: wrote matrix table with 791632 rows and 287 columns in 54 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/gnmd-part005.mt
2021-02-11 19:29:45 Hail: INFO: wrote matrix table with 657421 rows and 287 columns in 48 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/gnmd-part006.mt
2021-02-11 19:30:01 Hail: INFO: wrote matrix table with 736920 rows and 287 columns in 54 partitions to /net/archive/groups/plggneuromol/sportsme

In [21]:
genes = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/genecode_v32.ht')

genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))

hpo = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/hpo.tsv', impute = True, no_header=True)

start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])

genes = genes.key_by(genes.interval)

for idx, mt in enumerate(mts):
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/gnmd-'+names[idx]+'.mt')
    mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
    mt = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
    mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/hpo-final-'+names[idx]+'.mt')
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/hpo-final'+names[idx]+'.mt')

In [18]:
mts = []

for m in names:
    f = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/hpo-'+m+'.mt')
    mts.append(f)

In [49]:
mts = mts[0:9]+mts[10:]

In [50]:
mt_full = hl.MatrixTable.union_rows(*mts)

In [54]:
mt_full.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint.mt')

2021-02-11 19:48:32 Hail: INFO: wrote matrix table with 10751439 rows and 287 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint.mt


In [8]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint.mt')

In [5]:
mt.count()

(10751439, 287)

In [57]:
mt_subset = mt.sample_rows(0.001)

In [59]:
mt_subset.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-subset.mt')

2021-02-11 19:50:15 Hail: INFO: wrote matrix table with 10709 rows and 287 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-subset.mt


In [55]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-subset.mt')

first lets perform PCA on all the samples

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

mt_subset = mt_subset.annotate_cols(group = hl.cond(mt_subset.s.contains("B"), "sport", "gts"))

mt_subset = mt_subset.annotate_cols(scores = pcs[mt_subset.s].scores)

p = hl.plot.scatter(mt_subset.scores[0],
                    mt_subset.scores[1],
                    label=mt_subset.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

### Create a separate matrix with GTS

In [3]:
to_keep = ['S_7156','WGS_108','WGS_113','WGS_122','WGS_139','WGS_13','WGS_141','WGS_149','WGS_154','WGS_15','WGS_168','WGS_171a','WGS_182a','WGS_183',
           'WGS_196','WGS_200a','WGS_201','WGS_202a','WGS_22','WGS_57a','WGS_5','WGS_6827','WGS_6857','WGS_6880','WGS_6955','WGS_6958','WGS_7039','WGS_7050',
           'WGS_7117','WGS_7168','WGS_7176','WGS_76','WGS_77','WGS_81','WGS_82','WGS_83','WGS_86','WGS_87','WGS_88','WGS_8']

In [8]:
gts = mt.filter_cols(hl.set(to_keep).contains(mt.s), keep=True)

In [9]:
gts.count()

(10751439, 40)

In [10]:
gts_subset = gts.sample_rows(0.001)

In [11]:
gts_subset.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/gts-subset.mt')

2021-03-18 14:50:06 Hail: INFO: wrote matrix table with 10709 rows and 40 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/gts-subset.mt
    Total size: 5.60 MiB
    * Rows/entries: 5.60 MiB
    * Columns: 224.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  10 rows (8.52 KiB)


In [12]:
pc_rel = hl.pc_relate(gts_subset.GT, 0.001, k=1, statistics='kin')

2021-03-18 15:14:18 Hail: INFO: hwe_normalized_pca: running PCA using 6705 variants.
2021-03-18 15:14:57 Hail: INFO: pca: running PCA with 1 components...
2021-03-18 15:19:30 Hail: INFO: Wrote all 3 blocks of 10709 x 40 matrix with block size 4096.


In [13]:
pairs = pc_rel.filter(pc_rel['kin'] > 0.2)

In [14]:
related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j,
                                                       keep=False)

2021-03-18 15:19:32 Hail: INFO: wrote matrix with 2 rows and 10709 columns as 3 blocks of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-ZWF7qwu8GTzQluXGpF5brR.bm
2021-03-18 15:19:33 Hail: INFO: wrote matrix with 10709 rows and 40 columns as 3 blocks of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-G9Ly0ykYjQJVumF3Rx2IjR.bm
2021-03-18 15:19:34 Hail: INFO: wrote matrix with 40 rows and 40 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-R9YPy0EzWVawPXnFZP77Zl.bm
2021-03-18 15:19:34 Hail: INFO: wrote matrix with 40 rows and 40 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-CdQ9oinEgOBY8n76xKzODs.bm
2021-03-18 15:19:34 Hail: INFO: wrote matrix with 40 rows and 40 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-9wQMv87M56K9NOnxGjAAyy.bm
2021-03-18 15:19:34 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-03-18 15:19:35 Hail

In [16]:
related_samples_to_remove.show()

""
node
str


#### There are no related samples in this GTS cohort

In [65]:
gts.count()

10751439

In [20]:
gts.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/unrelated-gts.mt')

2021-03-18 16:11:30 Hail: INFO: wrote matrix table with 10751439 rows and 40 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/unrelated-gts-samples.mt
    Total size: 3.14 GiB
    * Rows/entries: 3.14 GiB
    * Columns: 224.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  6444 rows (5.47 MiB)
2021-03-18 16:12:36 Hail: INFO: wrote matrix table with 10751439 rows and 40 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/unrelated-gts.mt
    Total size: 3.14 GiB
    * Rows/entries: 3.14 GiB
    * Columns: 224.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  6444 rows (5.47 MiB)


### remove samples that are not controls

In [14]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint.mt')

In [15]:
mt.count()

(10751439, 287)

In [44]:
controls_to_keep = ['S_7212','S_7213','S_7214','S_7227','S_7236','S_7255','S_7237','S_7239','S_7241','S_7245','S_7246',
'S_7247','S_7229','S_7252','S_7254','WGS_147c','S_7261','S_7263','S_7269','S_7274','S_7291',
'S_7294','S_7306','S_7307','WGS_37b','WGS_37c','WGS_85b','WGS_7118','WGS_7120','WGS_7142','WGS_7143',
'WGS_7152','WGS_7153','WGS_163b','WGS_163d','S_7250','WGS_180b','WGS_6819','WGS_185c',
'WGS_6812','WGS_6814','WGS_D6813','WGS_D6815','462','468','475','476','477','478','479','490','492','494']

In [45]:
len(controls_to_keep)

53

In [19]:
sportsmen=['B102','B156','B24','B338','B382','B399','B427','B430','B431','B432','B433','B435','B436','B437','B438','B439','B441','B442','B443','B444','B445','B446','B448','B449','B450','B451','B452','B453','B454',
'B455','B456','B457','B458','B459','B461','B462','B463','B464','B465','B466','B467','B468','B469','B470','B471','B472','B473','B474','B475','B476','B477','B478','B479','B480','B481','B482','B483','B485','B486','B487','B488',
'B489','B490','B491','B492','B493','B494','B495','B496','B497','B498','B499','B501','B502','B503','B504','B505','B506','B507','B508','B509','B515','B518','B522','B523','B524','B525','B526','B527','B528','B529','B530','B531',
'B532','B533','B534','B535','B536','B537','B538','B539','B81']

In [26]:
to_keep = controls_to_keep + sportsmen

In [27]:
mt = mt.filter_cols(hl.set(to_keep).contains(mt.s), keep=True)

In [28]:
mt.count()

(10751439, 155)

In [29]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-healthy.mt')

2021-03-29 16:14:29 Hail: INFO: wrote matrix table with 10751439 rows and 155 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-healthy.mt
    Total size: 9.28 GiB
    * Rows/entries: 9.28 GiB
    * Columns: 691.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  6444 rows (9.05 MiB)


## extra control of genotype quality

In [15]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-healthy.mt')

In [29]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_all.ht')

rpmk = rpmk.annotate(larger_int = hl.if_else(rpmk.interval.start.position == 1,
                                             hl.locus(rpmk.interval.start.contig, (rpmk.interval.start.position), reference_genome='GRCh38'),
                                            hl.locus(rpmk.interval.start.contig, (rpmk.interval.start.position-1), reference_genome='GRCh38')))
                                        
                     

rpmk = rpmk.key_by(rpmk.larger_int)

rpmk = rpmk.select()

rpmk.write('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_expanded.ht')

rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_expanded.ht')

mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)

In [36]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-healthy-filtered.mt')

2021-03-30 10:05:28 Hail: INFO: wrote matrix table with 9824949 rows and 155 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-healthy-filtered.mt
    Total size: 8.47 GiB
    * Rows/entries: 8.47 GiB
    * Columns: 691.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (8.86 MiB)


In [66]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-healthy-filtered.mt')

In [67]:
mt.count()

(9824949, 155)

In [40]:
mt_subset = mt.sample_rows(0.001)
mt_subset.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-subset-healthy.mt')

2021-03-30 10:09:44 Hail: INFO: wrote matrix table with 9843 rows and 155 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-subset-healthy.mt
    Total size: 11.55 MiB
    * Rows/entries: 11.55 MiB
    * Columns: 691.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  15 rows (17.17 KiB)


### At this time we realised there was a mistake in genotyping of some of the samples. These are replaced in the matrix table at this point

In [48]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-subset-healthy.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-healthy-filtered.mt')

In [8]:
samples_to_remove = hl.set(["B522", "B506", "B507", "B508", "B509",
                            "B523", "B478", "B449"]) #sample B449 could not be corrected, thus it is permanently removed

In [9]:
mt = mt.filter_cols(
     samples_to_remove.contains(mt.s), keep=False)

In [10]:
mt_subset = mt_subset.filter_cols(
     samples_to_remove.contains(mt_subset.s), keep=False)

In [11]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-incomplete-lists.mt')
mt_subset.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-incomplete-lists-subset.mt')

2021-04-15 13:02:25 Hail: INFO: wrote matrix table with 9824949 rows and 147 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-incomplete-lists.mt
    Total size: 8.03 GiB
    * Rows/entries: 8.03 GiB
    * Columns: 659.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (8.43 MiB)
2021-04-15 13:02:42 Hail: INFO: wrote matrix table with 9843 rows and 147 columns in 2054 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-incomplete-lists-subset.mt
    Total size: 11.01 MiB
    * Rows/entries: 11.01 MiB
    * Columns: 659.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  15 rows (16.21 KiB)


In [6]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-incomplete-lists-subset.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-incomplete-lists.mt')

In [7]:
mt.count()

(9824949, 147)

In [ ]:
for f in ['B506','B478', 'B507', 'B508', 'B509', 'B522', 'B523']:
    hl.import_vcf('/net/archive/groups/plggneuromol/sportsmen-wgs/vcf/vcf-corrected/'+f+'.vcf.gz',
                  reference_genome='GRCh38',
                  force_bgz = True, array_elements_required = False).write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/'+f+'.mt')


2021-04-21 22:34:37 Hail: INFO: Coerced sorted dataset
2021-04-21 22:47:55 Hail: INFO: wrote matrix table with 3084456838 rows and 1 column in 147 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/B478.mt
    Total size: 52.06 GiB
    * Rows/entries: 52.06 GiB
    * Columns: 16.00 B
    * Globals: 11.00 B
    * Smallest partition: 19979803 rows (351.18 MiB)
    * Largest partition:  27959649 rows (456.71 MiB)
2021-04-21 23:19:21 Hail: INFO: wrote matrix table with 3084456838 rows and 1 column in 147 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/B478-filtered.mt
    Total size: 52.06 GiB
    * Rows/entries: 52.06 GiB
    * Columns: 16.00 B
    * Globals: 11.00 B
    * Smallest partition: 19979803 rows (351.18 MiB)
    * Largest partition:  27959649 rows (456.71 MiB)
2021-04-21 23:39:38 Hail: INFO: Coerced sorted dataset
2021-04-21 23:52:16 Hail: INFO: wrote matrix table with 3084402782 rows and 1 column in 141 partitions to /net/archive/groups/plggneuro

In [ ]:
for f in ['B506','B478', 'B507', 'B508', 'B509', 'B522', 'B523']:
    sample = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/'+f+'.mt')
    sample = sample.filter_rows(hl.is_defined(mt.index_rows(sample.row_key)), keep = True)
    sample.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/'+f+'-filtered.mt')

In [9]:
mts = []

for f in ['B506','B478', 'B507', 'B508', 'B509', 'B522', 'B523']:
    mts.append(hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/'+f+'-filtered.mt'))

In [59]:
final = mt

for idx, f in enumerate(mts):
    final = final.union_cols(f, row_join_type ='outer')
    final.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete'+str(idx)+'.mt')

2021-04-23 14:04:32 Hail: INFO: wrote matrix table with 9824949 rows and 148 columns in 2199 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete0.mt
    Total size: 10.29 GiB
    * Rows/entries: 10.29 GiB
    * Columns: 673.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (10.49 MiB)
2021-04-23 14:07:23 Hail: INFO: wrote matrix table with 9824949 rows and 149 columns in 2345 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete1.mt
    Total size: 10.31 GiB
    * Rows/entries: 10.31 GiB
    * Columns: 677.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (10.49 MiB)
2021-04-23 14:11:35 Hail: INFO: wrote matrix table with 9824949 rows and 150 columns in 2485 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete2.mt
    Total size: 10.33 GiB
    * Rows/entries: 10.33 GiB
    * Columns: 682.00 B
    * Globals:

FatalError: SparkException: Job aborted due to stage failure: Task 1116 in stage 14.0 failed 4 times, most recent failure: Lost task 1116.3 in stage 14.0 (TID 25871, 172.20.72.10, executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
Driver stacktrace:

Java stack trace:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 1116 in stage 14.0 failed 4 times, most recent failure: Lost task 1116.3 in stage 14.0 (TID 25871, 172.20.72.10, executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(D

In [11]:
final = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete3.mt')

In [5]:
final.count()

(9824949, 151)

In [12]:
for idx, f in enumerate(mts[4:]):
    final = final.union_cols(f, row_join_type ='outer')
    final.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete'+str(idx+4)+'.mt')

2021-04-23 14:31:14 Hail: INFO: wrote matrix table with 9824949 rows and 152 columns in 2776 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete4.mt
    Total size: 10.37 GiB
    * Rows/entries: 10.37 GiB
    * Columns: 690.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (10.59 MiB)
2021-04-23 14:34:19 Hail: INFO: wrote matrix table with 9824949 rows and 153 columns in 2918 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete5.mt
    Total size: 10.39 GiB
    * Rows/entries: 10.39 GiB
    * Columns: 693.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (10.59 MiB)
2021-04-23 14:38:53 Hail: INFO: wrote matrix table with 9824949 rows and 154 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to-delete6.mt
    Total size: 10.41 GiB
    * Rows/entries: 10.41 GiB
    * Columns: 697.00 B
    * Globals:

In [13]:
final.count()

(9824949, 154)

In [14]:
final.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-with-corrected-samples.mt')

2021-04-23 14:44:19 Hail: INFO: wrote matrix table with 9824949 rows and 154 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-with-corrected-samples.mt
    Total size: 8.14 GiB
    * Rows/entries: 8.14 GiB
    * Columns: 676.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (8.55 MiB)


In [4]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-with-corrected-samples.mt')

In [5]:
mt.count()

(9824949, 154)

In [17]:
mt_subset = mt_subset = mt.sample_rows(0.001)

In [18]:
mt_subset.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-with-corrected-samples-subset.mt')

2021-04-23 14:44:56 Hail: INFO: wrote matrix table with 9941 rows and 154 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-with-corrected-samples-subset.mt
    Total size: 13.80 MiB
    * Rows/entries: 13.80 MiB
    * Columns: 697.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  15 rows (20.72 KiB)


In [6]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-with-corrected-samples-subset.mt')

In [20]:
to_delete = ['476', '477', '478', 'WGS_6814', 'WGS_163d', 'S_7239', 'B461'] # this is to delete overly related samples + a sample that is outlier on PCA

In [21]:
mt_subset = mt_subset.filter_cols(hl.set(to_delete).contains(mt_subset.s), keep=False)

In [22]:
mt_subset.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-subset-unrelated.mt')

2021-04-23 14:45:21 Hail: INFO: wrote matrix table with 9941 rows and 147 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-subset-unrelated.mt
    Total size: 13.31 MiB
    * Rows/entries: 13.31 MiB
    * Columns: 669.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  15 rows (19.79 KiB)


In [51]:
pc_rel = hl.pc_relate(mt_subset.GT, 0.001, k=2, statistics='kin')

2021-04-23 11:51:02 Hail: INFO: hwe_normalized_pca: running PCA using 8353 variants.
2021-04-23 11:51:07 Hail: INFO: pca: running PCA with 2 components...
2021-04-23 12:18:26 Hail: INFO: Wrote all 3 blocks of 9841 x 143 matrix with block size 4096.


In [52]:
pairs = pc_rel.filter(pc_rel['kin'] > 0.23)

In [53]:
pairs.show()

2021-04-23 12:18:27 Hail: INFO: wrote matrix with 3 rows and 9841 columns as 3 blocks of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-7XbZ9EKjjtNbhQgUjOTNCU.bm
2021-04-23 12:18:27 Hail: INFO: wrote matrix with 9841 rows and 143 columns as 3 blocks of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-a4M9w8cThJnaI5rpH7eahB.bm
2021-04-23 12:18:27 Hail: INFO: wrote matrix with 143 rows and 143 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-MyBeJitdWsK9CqeXWjD2ps.bm
2021-04-23 12:18:27 Hail: INFO: wrote matrix with 143 rows and 143 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-cYECbxph3ZVrWhAr8G2QFh.bm
2021-04-23 12:18:27 Hail: INFO: wrote matrix with 143 rows and 143 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-xuJ8pyBqqjFZDBRcSw9kx5.bm
2021-04-23 12:18:27 Hail: INFO: Ordering unsorted dataset with network shuffle


,,
i,j,
s,s,kin
str,str,float64


In [9]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-with-corrected-samples.mt')

In [10]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-subset-unrelated.mt')

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt_subset.GT)

mt_subset = mt_subset.annotate_cols(group = hl.cond(mt_subset.s.contains("B"), "sport", "gts"))
mt_subset = mt_subset.annotate_cols(scores = pcs[mt_subset.s].scores)

2021-04-23 14:45:28 Hail: INFO: hwe_normalized_pca: running PCA using 8525 variants.
2021-04-23 14:45:35 Hail: INFO: pca: running PCA with 10 components...


In [ ]:
mt_subset.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/subset-with-pca.mt')

In [25]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/subset-with-pca.mt')

FatalError: HailException: MatrixTable and Table files are directories; path '/net/archive/groups/plggneuromol/sportsmen-wgs/mts/subset-with-pca.mt' is not a directory

Java stack trace:
is.hail.utils.HailException: MatrixTable and Table files are directories; path '/net/archive/groups/plggneuromol/sportsmen-wgs/mts/subset-with-pca.mt' is not a directory
	at is.hail.utils.ErrorHandling$class.fatal(ErrorHandling.scala:11)
	at is.hail.utils.package$.fatal(package.scala:77)
	at is.hail.expr.ir.RelationalSpec$.readMetadata(AbstractMatrixTableSpec.scala:32)
	at is.hail.expr.ir.RelationalSpec$.readReferences(AbstractMatrixTableSpec.scala:66)
	at is.hail.variant.ReferenceGenome$.fromHailDataset(ReferenceGenome.scala:596)
	at is.hail.variant.ReferenceGenome.fromHailDataset(ReferenceGenome.scala)
	at sun.reflect.GeneratedMethodAccessor49.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.jav

In [ ]:
p = hl.plot.scatter(mt_subset.scores[0],
                    mt_subset.scores[1],
                    label=mt_subset.s,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

In [ ]:
# should we delete sample B461? 
pcs = mt_subset.scores[1].collect()
samples = mt_subset.s.collect()

np.std(pcs)*6



#pcs[40]

# Sample B461 extendes more than 6 SDs so we will delete it - deleted above and PCA repeated

In [ ]:
mt.count()

In [11]:
to_delete = ['476', '477', '478', 'WGS_6814', 'WGS_163d', 'S_7239', 'B461'] 

In [12]:
mt = mt.filter_cols(hl.set(to_delete).contains(mt.s), keep=False)

In [13]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-unrelated-corrected.mt')

2021-04-23 15:31:57 Hail: INFO: wrote matrix table with 9824949 rows and 147 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-unrelated-corrected.mt
    Total size: 7.79 GiB
    * Rows/entries: 7.79 GiB
    * Columns: 648.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (8.16 MiB)


In [14]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-unrelated-corrected.mt')

In [15]:
mt.count()

(9824949, 147)

## april 2021 analyses

In [16]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-unrelated-corrected.mt')

In [17]:
mt = mt.filter_rows(mt.gnomad_v3.v3_nfe.AF < 0.05)

In [18]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005.mt')

2021-04-23 15:33:11 Hail: INFO: wrote matrix table with 4925533 rows and 147 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005.mt
    Total size: 3.38 GiB
    * Rows/entries: 3.38 GiB
    * Columns: 648.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4095 rows (3.74 MiB)


In [19]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005.mt')

In [20]:
genes = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/genecode_v32.ht')

genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))

hpo = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/hpo.tsv', impute = True, no_header=True)

start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])

genes = genes.key_by(genes.interval)

mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
  
    
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-genes.mt')

2021-04-23 15:33:11 Hail: INFO: Reading table to impute column types
2021-04-23 15:33:12 Hail: INFO: Finished type imputation
  Loading field 'f0' as type str (imputed)
  Loading field 'f1' as type str (imputed)
2021-04-23 15:33:13 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-23 15:33:14 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-23 15:33:15 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-23 15:33:15 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-23 15:33:16 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-23 15:33:17 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-04-23 15:34:32 Hail: INFO: wrote matrix table with 4925533 rows and 147 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-genes.mt
    Total size: 3.49 GiB
    * Rows/entries: 3.49 GiB
    * Columns: 648.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows

In [21]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-genes.mt')

In [22]:
mt.count()

(4925533, 147)

In [23]:
mt = mt.filter_rows(mt.within_gene == hl.empty_array(hl.tstr), keep = False)

In [24]:
mt.count()

(3191612, 147)

In [25]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic.mt')

2021-04-23 15:35:23 Hail: INFO: wrote matrix table with 3191612 rows and 147 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic.mt
    Total size: 2.39 GiB
    * Rows/entries: 2.39 GiB
    * Columns: 648.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3841 rows (4.51 MiB)


In [66]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic.mt')

In [67]:
mt.count()

(3191612, 147)

## lets further filter variants - higher quality qenotypes only

In [28]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_all.ht')

contig_len = hl.contig_length(rpmk.interval.end.contig, reference_genome='GRCh38')
end_pos = rpmk.interval.end.position
start_pos = rpmk.interval.start.position
contig = rpmk.interval.start.contig

rpmk = rpmk.annotate(larger_int_start = hl.if_else(start_pos == 1,
                                                   hl.locus(contig, start_pos, reference_genome='GRCh38'),
                                                   hl.locus(contig, start_pos-1, reference_genome='GRCh38')),
                     larger_int_end = hl.if_else(end_pos == contig_len,
                                                 hl.locus(contig, end_pos, reference_genome='GRCh38'),
                                                 hl.locus(contig, end_pos+1, reference_genome='GRCh38')),
                     larger_int_start_2 = hl.if_else((start_pos == 1) | (start_pos == 2),
                                                   hl.locus(contig, start_pos, reference_genome='GRCh38'),
                                                   hl.locus(contig, start_pos-2, reference_genome='GRCh38')),
                     larger_int_end_2 = hl.if_else((end_pos == contig_len) | (end_pos == (contig_len-1)),
                                                 hl.locus(contig, end_pos, reference_genome='GRCh38'),
                                                 hl.locus(contig, end_pos+2, reference_genome='GRCh38')))

In [25]:
rpmk.write('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')

2021-03-30 20:23:23 Hail: INFO: wrote table with 5520017 rows in 48 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht
    Total size: 214.57 MiB
    * Rows: 214.57 MiB
    * Globals: 11.00 B
    * Smallest partition: 51769 rows (2.42 MiB)
    * Largest partition:  153107 rows (5.90 MiB)


In [ ]:
#for some reason a loop doesn't work here
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_start'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered0.mt')
    
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_end'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered1.mt')

rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_start_2'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered2.mt')



2021-04-23 15:35:26 Hail: INFO: Coerced sorted dataset
2021-04-23 15:36:08 Hail: INFO: wrote matrix table with 3191612 rows and 147 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered0.mt
    Total size: 3.19 GiB
    * Rows/entries: 3.19 GiB
    * Columns: 669.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3841 rows (5.60 MiB)
2021-04-23 15:36:11 Hail: INFO: Coerced sorted dataset
2021-04-23 15:36:13 Hail: INFO: Ordering unsorted dataset with network shuffle


In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered2.mt')

In [ ]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_end_2'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered.mt')

    

In [68]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered.mt')

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
mt = mt.annotate_cols(group = hl.if_else(mt.s.contains("B"), "sport", "control"))

In [ ]:
mt = mt.filter_rows(mt.variant_qc.dp_stats.mean > 3)

In [ ]:
mt = mt.annotate_rows(controls_non_ref = hl.agg.filter((mt.group == 'control'), hl.agg.count_where(mt.GT.is_non_ref())),
                      sports_non_ref = hl.agg.filter((mt.group == 'sport'), hl.agg.count_where(mt.GT.is_non_ref())))
mt = mt.annotate_rows(controls_ref = hl.agg.filter((mt.group == 'control'), hl.agg.count_where(mt.GT.is_hom_ref())),
                      sports_ref = hl.agg.filter((mt.group == 'sport'), hl.agg.count_where(mt.GT.is_hom_ref())))


In [ ]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

## analysis of all protein coding genes

In [208]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [209]:
mt.count()

(3154875, 147)

In [210]:
mt = mt.filter_rows((mt.controls_non_ref == 0)|(mt.sports_non_ref == 0), keep=False)

In [211]:
go_genes = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/gts_gene_lists/human-genes-with-GO-and-symbols')

2021-04-23 19:48:31 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)


In [212]:
all_genes = go_genes['UniProtKB Gene Name symbol'].collect()

In [213]:
len(all_genes)

23804

In [214]:
mt = mt.filter_rows(hl.any(lambda x: hl.literal(all_genes).contains(x), mt.within_gene))

In [215]:
mt = mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))

In [ ]:
to_test = mt.rows()
to_test = to_test.select(to_test.controls_non_ref, to_test.sports_non_ref, to_test.controls_ref, to_test.sports_ref)

to_test.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_005.ht')

In [216]:
to_test = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_005.ht')

In [217]:
to_test.count()

587838

In [218]:
0.05/to_test.count()

8.505744780024429e-08

In [219]:
to_test = to_test.annotate(fisher = hl.fisher_exact_test(hl.int32(to_test.controls_non_ref), 
                                                         hl.int32(to_test.controls_ref),
                                                         hl.int32(to_test.sports_non_ref),
                                                         hl.int32(to_test.sports_ref)))

In [220]:
to_test = to_test.filter(to_test.fisher.p_value < 0.05/to_test.count())

In [221]:
to_test.count()

16

In [222]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [223]:
mt_test = mt.filter_rows(hl.is_defined(to_test[mt.row_key]), keep = True)
mt_test = mt_test.annotate_rows(fisher = to_test[mt_test.row_key]['fisher'])

In [224]:
mt_test = mt_test.select_rows(rsid = mt_test.gnomad_v3.rsid,
                                 gnomAD_v3_1_allele_frequency = mt_test.gnomad_v3.v3_nfe.AF,
                                 gnomAD_v3_1_homozygote_count = mt_test.gnomad_v3.v3_nfe.homozygote_count,
                                 predicted_most_severe_consequence = mt_test.gnomad_v3.most_severe_consequence,
                                 gene = mt_test.within_gene,
                                 hpo = mt_test.hpo,
                                 controls_non_ref = mt_test.controls_non_ref,
                                 sports_non_ref = mt_test.sports_non_ref,
                                 controls_ref = mt_test.controls_ref,
                                 sports_ref = mt_test.sports_ref,
                                 percent_controls_non_ref = mt_test.controls_non_ref/47*100,
                                 percent_sports_non_ref = mt_test.sports_non_ref/100*100,
                                 percent_controls_ref = mt_test.controls_ref/46*100,
                                 percent_sports_ref = mt_test.sports_ref/100*100,
                                 percent_sports_genotyped = (mt_test.sports_non_ref + mt_test.sports_ref)/100*100,
                                 percent_controls_genotyped = (mt_test.controls_non_ref + mt_test.controls_ref)/47*100,
                                 fisher = mt_test.fisher.p_value,
                                 )

In [225]:
mt_test = mt_test.annotate_rows(
    samples_het = hl.agg.filter(mt_test.GT.is_het(), hl.agg.collect(mt_test.s)),
    samples_hom_non_ref = hl.agg.filter(mt_test.GT.is_hom_var(), hl.agg.collect(mt_test.s)))

In [226]:
mt_test.count()

(16, 147)

In [227]:
mt_test.aggregate_cols(hl.agg.counter(mt_test.group))

{'control': 47, 'sport': 100}

In [228]:
### annotate with frequency in GTS:

gts = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/unrelated-gts.mt')

gts = gts.annotate_rows(gts_non_ref = hl.agg.count_where(gts.GT.is_non_ref()),
                        gts_ref = hl.agg.count_where(gts.GT.is_hom_ref()))

gts = gts.rows()

gts = gts.select(gts_non_ref = gts.gts_non_ref,
                 gts_ref = gts.gts_ref,
                 percent_gts_non_ref = gts.gts_non_ref/40*100,
                 percent_gts_ref = gts.gts_ref/40*100,
                 percent_gts_genotyped = (gts.gts_non_ref + gts.gts_ref)/40*100)

mt_test = mt_test.annotate_rows(gts_non_ref = gts.index(mt_test.row_key).gts_non_ref,
                                gts_ref = gts.index(mt_test.row_key).gts_ref,
                                percent_gts_non_ref = gts.index(mt_test.row_key).percent_gts_non_ref,
                                percent_gts_ref = gts.index(mt_test.row_key).percent_gts_ref,
                                percent_gts_genotyped = gts.index(mt_test.row_key).percent_gts_genotyped)



to_export = mt_test.rows()

to_export = to_export.to_pandas()

In [229]:
to_export.columns

Index(['locus.contig', 'locus.position', 'alleles', 'rsid',
       'gnomAD_v3_1_allele_frequency', 'gnomAD_v3_1_homozygote_count',
       'predicted_most_severe_consequence', 'gene', 'hpo', 'controls_non_ref',
       'sports_non_ref', 'controls_ref', 'sports_ref',
       'percent_controls_non_ref', 'percent_sports_non_ref',
       'percent_controls_ref', 'percent_sports_ref',
       'percent_sports_genotyped', 'percent_controls_genotyped', 'fisher',
       'samples_het', 'samples_hom_non_ref', 'gts_non_ref', 'gts_ref',
       'percent_gts_non_ref', 'percent_gts_ref', 'percent_gts_genotyped'],
      dtype='object')

In [230]:
to_export.shape

(16, 27)

In [231]:
to_export

,locus.contig,locus.position,alleles,rsid,gnomAD_v3_1_allele_frequency,gnomAD_v3_1_homozygote_count,predicted_most_severe_consequence,gene,hpo,controls_non_ref,...,percent_sports_genotyped,percent_controls_genotyped,fisher,samples_het,samples_hom_non_ref,gts_non_ref,gts_ref,percent_gts_non_ref,percent_gts_ref,percent_gts_genotyped
0,chr1,237515763,"[C, G]",rs1436318549,0.016573,2,intron_variant,[RYR2],"[[HP:0001699:Sudden death, HP:0011663:Right ve...",4,...,89.0,100.000000,1.721915e-08,"[B338, B432, B433, B435, B436, B437, B442, B44...","[B102, B156, B439, B444, B448, B450, B453, B45...",3,36,7.500000,90.0,97.5
1,chr2,95935226,"[T, C]",rs550694093,0.005094,0,intron_variant,[ANKRD36C],[None],5,...,98.0,100.000000,4.146488e-10,"[492, B24, B338, B399, B427, B430, B431, B432,...",[],0,39,0.000000,97.5,97.5
2,chr2,114871252,"[G, C]",rs1438743709,0.004593,32,intron_variant,[DPP10],[None],19,...,94.0,100.000000,5.655587e-09,"[B462, S_7213, S_7214, S_7227, S_7229, S_7241,...","[S_7237, WGS_37c]",10,28,25.000000,70.0,95.0
3,chr3,75686761,"[C, G]",rs186911967,0.000016,0,intron_variant,"[LINC00960, ZNF717]",[None],1,...,97.0,100.000000,6.353729e-15,"[479, B24, B382, B399, B427, B430, B431, B432,...",[],1,38,2.500000,95.0,97.5
4,chr3,75686786,"[C, A]",rs182818770,0.000034,0,intron_variant,"[LINC00960, ZNF717]",[None],1,...,94.0,100.000000,6.914127e-14,"[479, B156, B24, B427, B431, B432, B433, B435,...",[],1,38,2.500000,95.0,97.5
5,chr6,157314313,"[C, G]",None,0.044423,0,intron_variant,[TMEM242],[None],27,...,93.0,100.000000,4.059197e-13,"[468, 475, 490, B468, B471, B487, S_7212, S_72...",[],16,23,40.000000,57.5,97.5
6,chr7,100956443,"[C, A]",rs1235870137,0.025931,0,missense_variant,[MUC3A],[None],28,...,93.0,100.000000,2.150403e-10,"[B427, B432, B439, B469, B474, B481, B483, B53...",[],26,14,65.000000,35.0,100.0
7,chr10,125897078,"[G, A]",None,0.025079,0,intron_variant,[FANK1],[None],1,...,97.0,100.000000,1.659989e-12,"[B102, B156, B338, B382, B427, B430, B431, B43...",[],1,38,2.500000,95.0,97.5
8,chr10,125902761,"[G, A]",rs59428255,0.030375,0,intron_variant,[FANK1],[None],5,...,96.0,100.000000,2.696115e-08,"[492, B102, B382, B399, B427, B430, B432, B433...",[],0,39,0.000000,97.5,97.5
9,chr10,125924980,"[G, A]",rs112222676,0.000492,0,intron_variant,[FANK1],[None],2,...,95.0,100.000000,1.505063e-08,"[490, 492, B156, B24, B338, B382, B399, B427, ...",[],0,39,0.000000,97.5,97.5


In [232]:
to_export.shape

(16, 27)

In [233]:
to_export.to_csv('filtered_variants_sportsmen_23_24_2020')

### genes enriched in muscle: https://www.proteinatlas.org/search/tissue_category_rna%3Askeletal+muscle%3BTissue+enriched+AND+sort_by%3Atissue+specific+score+AND+show_columns%3Agroupenriched

In [234]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [235]:
mt.count()

(3154875, 147)

In [236]:
mt = mt.filter_rows((mt.controls_non_ref == 0)|(mt.sports_non_ref == 0), keep=False)

In [237]:
muscle_genes = hl.import_table('./gene-lists-and-results/tissue_category_rna_skeletal.tsv')

2021-04-23 19:50:03 Hail: INFO: Loading 292 fields. Counts by type:
  str: 292


In [238]:
genes = muscle_genes['Gene'].collect()

In [239]:
len(genes)

907

In [240]:
mt = mt.filter_rows(hl.any(lambda x: hl.literal(genes).contains(x), mt.within_gene))

In [241]:
mt = mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))

In [84]:
to_test = mt.rows()
to_test = to_test.select(to_test.controls_non_ref, to_test.sports_non_ref, to_test.controls_ref, to_test.sports_ref)

to_test.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_muscle_005.ht')

2021-04-23 19:06:16 Hail: INFO: wrote table with 33484 rows in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_muscle_005.ht
    Total size: 712.53 KiB
    * Rows: 712.52 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  927 rows (15.40 KiB)


In [242]:
to_test = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_muscle_005.ht')

In [243]:
to_test.count()

33484

In [244]:
0.3/to_test.count()

8.959503046231036e-06

In [245]:
to_test = to_test.annotate(fisher = hl.fisher_exact_test(hl.int32(to_test.controls_non_ref), 
                                                         hl.int32(to_test.controls_ref),
                                                         hl.int32(to_test.sports_non_ref),
                                                         hl.int32(to_test.sports_ref)))

In [246]:
to_test = to_test.filter(to_test.fisher.p_value < 0.3/to_test.count())

In [247]:
to_test.count() #so I skip this

0

### genes elevated in the brain: https://www.proteinatlas.org/search/tissue_category_rna%3Abrain%3BTissue+enriched%2CGroup+enriched%2CTissue+enhanced+AND+sort_by%3Atissue+specific+score+AND+show_columns%3Agroupenriched

In [248]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [249]:
mt.count()

(3154875, 147)

In [250]:
mt = mt.filter_rows((mt.controls_non_ref == 0)|(mt.sports_non_ref == 0), keep=False)

In [251]:
brain_genes = hl.import_table('./gene-lists-and-results/tissue_category_rna_brain_Tissue.tsv')

2021-04-23 19:50:10 Hail: INFO: Loading 292 fields. Counts by type:
  str: 292


In [252]:
genes = brain_genes['Gene'].collect()

In [253]:
len(genes)

2587

In [254]:
mt = mt.filter_rows(hl.any(lambda x: hl.literal(genes).contains(x), mt.within_gene))

In [255]:
mt = mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))

In [131]:
to_test = mt.rows()
to_test = to_test.select(to_test.controls_non_ref, to_test.sports_non_ref, to_test.controls_ref, to_test.sports_ref)

to_test.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_brain_005.ht')

2021-04-23 19:14:45 Hail: INFO: wrote table with 167346 rows in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_brain_005.ht
    Total size: 3.06 MiB
    * Rows: 3.06 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  1077 rows (17.62 KiB)


In [256]:
to_test = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_brain_005.ht')

In [257]:
to_test.count()

167346

In [258]:
0.05/to_test.count()

2.9878216389994386e-07

In [259]:
to_test = to_test.annotate(fisher = hl.fisher_exact_test(hl.int32(to_test.controls_non_ref), 
                                                         hl.int32(to_test.controls_ref),
                                                         hl.int32(to_test.sports_non_ref),
                                                         hl.int32(to_test.sports_ref)))

In [260]:
to_test = to_test.filter(to_test.fisher.p_value < 0.05/to_test.count())

In [261]:
to_test.count()

5

In [262]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [263]:
mt_test = mt.filter_rows(hl.is_defined(to_test[mt.row_key]), keep = True)
mt_test = mt_test.annotate_rows(fisher = to_test[mt_test.row_key]['fisher'])

In [264]:
mt_test = mt_test.select_rows(rsid = mt_test.gnomad_v3.rsid,
                                 gnomAD_v3_1_allele_frequency = mt_test.gnomad_v3.v3_nfe.AF,
                                 gnomAD_v3_1_homozygote_count = mt_test.gnomad_v3.v3_nfe.homozygote_count,
                                 predicted_most_severe_consequence = mt_test.gnomad_v3.most_severe_consequence,
                                 gene = mt_test.within_gene,
                                 hpo = mt_test.hpo,
                                 controls_non_ref = mt_test.controls_non_ref,
                                 sports_non_ref = mt_test.sports_non_ref,
                                 controls_ref = mt_test.controls_ref,
                                 sports_ref = mt_test.sports_ref,
                                 percent_controls_non_ref = mt_test.controls_non_ref/47*100,
                                 percent_sports_non_ref = mt_test.sports_non_ref/100*100,
                                 percent_controls_ref = mt_test.controls_ref/47*100,
                                 percent_sports_ref = mt_test.sports_ref/100*100,
                                 percent_sports_genotyped = (mt_test.sports_non_ref + mt_test.sports_ref)/100*100,
                                 percent_controls_genotyped = (mt_test.controls_non_ref + mt_test.controls_ref)/47*100,
                                 fisher = mt_test.fisher.p_value,
                                 )

In [265]:
mt_test = mt_test.annotate_rows(
    samples_het = hl.agg.filter(mt_test.GT.is_het(), hl.agg.collect(mt_test.s)),
    samples_hom_non_ref = hl.agg.filter(mt_test.GT.is_hom_var(), hl.agg.collect(mt_test.s)))

In [266]:
mt_test.count()

(5, 147)

In [267]:
mt_test.aggregate_cols(hl.agg.counter(mt_test.group))

{'control': 47, 'sport': 100}

In [268]:
### annotate with frequency in GTS:

gts = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/unrelated-gts.mt')

gts = gts.annotate_rows(gts_non_ref = hl.agg.count_where(gts.GT.is_non_ref()),
                        gts_ref = hl.agg.count_where(gts.GT.is_hom_ref()))

gts = gts.rows()

gts = gts.select(gts_non_ref = gts.gts_non_ref,
                 gts_ref = gts.gts_ref,
                 percent_gts_non_ref = gts.gts_non_ref/40*100,
                 percent_gts_ref = gts.gts_ref/40*100,
                 percent_gts_genotyped = (gts.gts_non_ref + gts.gts_ref)/40*100)

mt_test = mt_test.annotate_rows(gts_non_ref = gts.index(mt_test.row_key).gts_non_ref,
                                gts_ref = gts.index(mt_test.row_key).gts_ref,
                                percent_gts_non_ref = gts.index(mt_test.row_key).percent_gts_non_ref,
                                percent_gts_ref = gts.index(mt_test.row_key).percent_gts_ref,
                                percent_gts_genotyped = gts.index(mt_test.row_key).percent_gts_genotyped)

In [269]:
to_export = mt_test.rows()

to_export = to_export.to_pandas()

In [270]:
to_export.columns

Index(['locus.contig', 'locus.position', 'alleles', 'rsid',
       'gnomAD_v3_1_allele_frequency', 'gnomAD_v3_1_homozygote_count',
       'predicted_most_severe_consequence', 'gene', 'hpo', 'controls_non_ref',
       'sports_non_ref', 'controls_ref', 'sports_ref',
       'percent_controls_non_ref', 'percent_sports_non_ref',
       'percent_controls_ref', 'percent_sports_ref',
       'percent_sports_genotyped', 'percent_controls_genotyped', 'fisher',
       'samples_het', 'samples_hom_non_ref', 'gts_non_ref', 'gts_ref',
       'percent_gts_non_ref', 'percent_gts_ref', 'percent_gts_genotyped'],
      dtype='object')

In [271]:
to_export.shape

(5, 27)

In [272]:
to_export

,locus.contig,locus.position,alleles,rsid,gnomAD_v3_1_allele_frequency,gnomAD_v3_1_homozygote_count,predicted_most_severe_consequence,gene,hpo,controls_non_ref,...,percent_sports_genotyped,percent_controls_genotyped,fisher,samples_het,samples_hom_non_ref,gts_non_ref,gts_ref,percent_gts_non_ref,percent_gts_ref,percent_gts_genotyped
0,chr2,114871252,"[G, C]",rs1438743709,0.004593,32,intron_variant,[DPP10],[None],19,...,94.0,100.000000,5.655587e-09,"[B462, S_7213, S_7214, S_7227, S_7229, S_7241,...","[S_7237, WGS_37c]",10,28,25.0,70.0,95.0
1,chr17,65139057,"[A, G]",None,0.040370,182,intron_variant,[RGS9],"[[HP:0000613:Photophobia, HP:0000505:Visual im...",5,...,34.0,95.744682,3.981673e-09,"[B477, S_7237, WGS_180b]","[B102, B156, B24, B382, B399, B427, B432, B437...",0,37,0.0,92.5,92.5
2,chr20,25773860,"[T, C]",rs201133036,0.010995,0,intron_variant,[FAM182B],[None],3,...,94.0,97.872337,1.585263e-07,"[B102, B338, B382, B399, B430, B439, B441, B44...",[],2,37,5.0,92.5,97.5
3,chr20,25776325,"[A, T]",rs1198672321,0.040598,0,intron_variant,[FAM182B],[None],30,...,94.0,97.872337,1.359509e-07,"[475, 479, 490, 492, B156, B24, B427, B439, B4...",[],30,9,75.0,22.5,97.5
4,chr22,20255642,"[A, C]",None,0.000214,0,intron_variant,[RTN4R],"[[HP:0000006:Autosomal dominant inheritance, H...",15,...,93.0,97.872337,1.284009e-07,"[B472, S_7212, S_7213, S_7214, S_7229, S_7236,...",[],0,38,0.0,95.0,95.0


In [273]:
to_export.to_csv('brain_variants_sportsmen')

### genes elevated in lungs: https://www.proteinatlas.org/search/tissue_category_rna%3Alung%3BTissue+enriched%2CGroup+enriched%2CTissue+enhanced+AND+sort_by%3Atissue+specific+score+AND+show_columns%3Agroupenriched

In [157]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [158]:
mt.count()

(3154875, 147)

In [147]:
mt = mt.filter_rows((mt.controls_non_ref == 0)|(mt.sports_non_ref == 0), keep=False)

In [159]:
lung_genes = hl.import_table('./gene-lists-and-results/tissue_category_rna_lung_Tissue.tsv')

2021-04-23 19:35:14 Hail: INFO: Loading 292 fields. Counts by type:
  str: 292


In [178]:
genes = lung_genes['Gene'].collect()

In [161]:
len(genes)

239

In [179]:
mt = mt.filter_rows(hl.any(lambda x: hl.literal(genes).contains(x), mt.within_gene))

In [164]:
to_test = mt.rows()
to_test = to_test.select(to_test.controls_non_ref, to_test.sports_non_ref, to_test.controls_ref, to_test.sports_ref)

to_test.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_lung_005.ht')

2021-04-23 19:36:35 Hail: INFO: wrote table with 25923 rows in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_lung_005.ht
    Total size: 538.17 KiB
    * Rows: 538.16 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  1146 rows (19.38 KiB)


In [180]:
to_test = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_lung_005.ht')

In [181]:
to_test.count()

25923

In [167]:
0.3/to_test.count()

1.1572734637194768e-05

In [168]:
to_test = to_test.annotate(fisher = hl.fisher_exact_test(hl.int32(to_test.controls_non_ref), 
                                                         hl.int32(to_test.controls_ref),
                                                         hl.int32(to_test.sports_non_ref),
                                                         hl.int32(to_test.sports_ref)))

In [170]:
to_test = to_test.filter(to_test.fisher.p_value < 0.05/to_test.count())

In [171]:
to_test.count()

21

In [172]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [173]:
mt_test = mt.filter_rows(hl.is_defined(to_test[mt.row_key]), keep = True)
mt_test = mt_test.annotate_rows(fisher = to_test[mt_test.row_key]['fisher'])

In [174]:
mt_test = mt_test.select_rows(rsid = mt_test.gnomad_v3.rsid,
                                 gnomAD_v3_1_allele_frequency = mt_test.gnomad_v3.v3_nfe.AF,
                                 gnomAD_v3_1_homozygote_count = mt_test.gnomad_v3.v3_nfe.homozygote_count,
                                 predicted_most_severe_consequence = mt_test.gnomad_v3.most_severe_consequence,
                                 gene = mt_test.within_gene,
                                 hpo = mt_test.hpo,
                                 controls_non_ref = mt_test.controls_non_ref,
                                 sports_non_ref = mt_test.sports_non_ref,
                                 controls_ref = mt_test.controls_ref,
                                 sports_ref = mt_test.sports_ref,
                                 percent_controls_non_ref = mt_test.controls_non_ref/47*100,
                                 percent_sports_non_ref = mt_test.sports_non_ref/100*100,
                                 percent_controls_ref = mt_test.controls_ref/47*100,
                                 percent_sports_ref = mt_test.sports_ref/100*100,
                                 percent_sports_genotyped = (mt_test.sports_non_ref + mt_test.sports_ref)/100*100,
                                 percent_controls_genotyped = (mt_test.controls_non_ref + mt_test.controls_ref)/47*100,
                                 fisher = mt_test.fisher.p_value,
                                 )

In [175]:
mt_test = mt_test.filter_rows(hl.is_snp(mt_test.alleles[0], mt_test.alleles[1]))

In [176]:
mt_test = mt_test.annotate_rows(
    samples_het = hl.agg.filter(mt_test.GT.is_het(), hl.agg.collect(mt_test.s)),
    samples_hom_non_ref = hl.agg.filter(mt_test.GT.is_hom_var(), hl.agg.collect(mt_test.s)))

In [177]:
mt_test.count()

(12, 147)

In [182]:
### annotate with frequency in GTS:

gts = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/unrelated-gts.mt')

gts = gts.annotate_rows(gts_non_ref = hl.agg.count_where(gts.GT.is_non_ref()),
                        gts_ref = hl.agg.count_where(gts.GT.is_hom_ref()))

gts = gts.rows()

gts = gts.select(gts_non_ref = gts.gts_non_ref,
                 gts_ref = gts.gts_ref,
                 percent_gts_non_ref = gts.gts_non_ref/40*100,
                 percent_gts_ref = gts.gts_ref/40*100,
                 percent_gts_genotyped = (gts.gts_non_ref + gts.gts_ref)/40*100)

mt_test = mt_test.annotate_rows(gts_non_ref = gts.index(mt_test.row_key).gts_non_ref,
                                gts_ref = gts.index(mt_test.row_key).gts_ref,
                                percent_gts_non_ref = gts.index(mt_test.row_key).percent_gts_non_ref,
                                percent_gts_ref = gts.index(mt_test.row_key).percent_gts_ref,
                                percent_gts_genotyped = gts.index(mt_test.row_key).percent_gts_genotyped)

In [183]:
to_export = mt_test.rows()

to_export = to_export.to_pandas()

In [184]:
to_export.columns

Index(['locus.contig', 'locus.position', 'alleles', 'rsid',
       'gnomAD_v3_1_allele_frequency', 'gnomAD_v3_1_homozygote_count',
       'predicted_most_severe_consequence', 'gene', 'hpo', 'controls_non_ref',
       'sports_non_ref', 'controls_ref', 'sports_ref',
       'percent_controls_non_ref', 'percent_sports_non_ref',
       'percent_controls_ref', 'percent_sports_ref',
       'percent_sports_genotyped', 'percent_controls_genotyped', 'fisher',
       'samples_het', 'samples_hom_non_ref', 'gts_non_ref', 'gts_ref',
       'percent_gts_non_ref', 'percent_gts_ref', 'percent_gts_genotyped'],
      dtype='object')

In [185]:
to_export.shape

(12, 27)

In [186]:
to_export

,locus.contig,locus.position,alleles,rsid,gnomAD_v3_1_allele_frequency,gnomAD_v3_1_homozygote_count,predicted_most_severe_consequence,gene,hpo,controls_non_ref,...,percent_sports_genotyped,percent_controls_genotyped,fisher,samples_het,samples_hom_non_ref,gts_non_ref,gts_ref,percent_gts_non_ref,percent_gts_ref,percent_gts_genotyped
0,chr6,32525558,"[A, C]",None,0.027921,0,intron_variant,[HLA-DRB5],[None],20,...,28.000000,85.106384,1.806750e-06,"[S_7212, S_7229, S_7246, S_7252, S_7261, S_726...","[S_7245, S_7247, S_7255, S_7294, WGS_163b, WGS...",9,20,22.500000,50.000000,72.5
1,chr6,32525559,"[T, C]",rs201618469,0.027942,0,intron_variant,[HLA-DRB5],[None],18,...,28.000000,85.106384,1.083374e-05,"[S_7212, S_7229, S_7246, S_7252, S_7261, S_726...","[S_7245, S_7247, S_7255, S_7294, WGS_163b, WGS...",9,20,22.500000,50.000000,72.5
2,chr6,32529071,"[C, T]",rs796985052,0.047066,0,intron_variant,[HLA-DRB5],[None],22,...,27.000002,89.361702,9.174138e-07,"[S_7212, S_7213, S_7229, S_7245, S_7247, S_725...",[],18,21,45.000000,52.499996,97.5
3,chr6,32529075,"[C, A]",rs1019319406,0.014959,0,intron_variant,[HLA-DRB5],[None],22,...,27.000002,89.361702,9.174138e-07,"[S_7212, S_7229, S_7236, S_7252, S_7255, S_726...",[],21,18,52.499996,45.000000,97.5
4,chr6,32529084,"[T, A]",rs965122936,0.016525,0,intron_variant,[HLA-DRB5],[None],23,...,27.000002,89.361702,5.866651e-07,"[S_7212, S_7229, S_7236, S_7246, S_7252, S_725...",[],20,19,50.000000,47.500000,97.5
5,chr6,32529099,"[G, A]",None,0.015727,0,intron_variant,[HLA-DRB5],[None],24,...,26.000000,89.361702,2.528710e-07,"[S_7212, S_7229, S_7236, S_7246, S_7252, S_725...",[],21,18,52.499996,45.000000,97.5
6,chr6,32529948,"[A, T]",rs114292540,0.005315,0,intron_variant,[HLA-DRB5],[None],22,...,38.000000,93.617020,6.675157e-08,"[S_7212, S_7229, S_7246, S_7261, S_7263, S_727...",[],20,20,50.000000,50.000000,100.0
7,chr6,32529983,"[A, T]",rs115098928,0.003106,0,intron_variant,[HLA-DRB5],[None],19,...,44.000000,93.617020,3.162186e-07,"[S_7212, S_7229, S_7246, S_7252, S_7261, S_726...",[],18,22,45.000000,55.000000,100.0
8,chr6,32530090,"[C, T]",rs752268643,0.034809,0,intron_variant,[HLA-DRB5],[None],19,...,46.000000,93.617020,9.810277e-08,"[S_7212, S_7229, S_7246, S_7252, S_7255, S_726...",[],12,24,30.000002,60.000004,90.0
9,chr6,32530133,"[T, C]",rs139583918,0.025449,0,missense_variant,[HLA-DRB5],[None],19,...,44.000000,93.617020,3.162186e-07,"[S_7212, S_7229, S_7246, S_7252, S_7255, S_726...",[],12,25,30.000002,62.500000,92.5


## rare variants

In [274]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [275]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [276]:
mt = mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))

In [277]:
mt = mt.annotate_cols(group = hl.cond(mt.s.contains("B"), "sport", "control"))

/net/software/local/python/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) cond. (Replaced by hl.if_else) -- Deprecated since version 0.2.59.
  """Entry point for launching an IPython kernel.


In [278]:
mt = mt.filter_rows((mt.gnomad_v3.v3_nfe.AF < 0.0001) & (mt.gnomad_v3.v3_nfe.AF > 0))

In [279]:
mt = mt.annotate_rows(controls_non_ref = hl.agg.filter((mt.group == 'control'), hl.agg.count_where(mt.GT.is_non_ref())),
                      sports_non_ref = hl.agg.filter((mt.group == 'sport'), hl.agg.count_where(mt.GT.is_non_ref())))

In [280]:
mt = mt.select_rows(rsid = mt.gnomad_v3.rsid,
                                 gnomAD_v3_1_allele_frequency = mt.gnomad_v3.v3_nfe.AF,
                                 gnomAD_v3_1_homozygote_count = mt.gnomad_v3.v3_nfe.homozygote_count,
                                 predicted_most_severe_consequence = mt.gnomad_v3.most_severe_consequence,
                                 gene = mt.within_gene,
                                 hpo = mt.hpo,
                                 controls_non_ref = mt.controls_non_ref,
                                 sports_non_ref = mt.sports_non_ref)

mt = mt.annotate_rows(
    samples_het = hl.agg.filter(mt.GT.is_het(), hl.agg.collect(mt.s)),
    samples_hom_non_ref = hl.agg.filter(mt.GT.is_hom_var(), hl.agg.collect(mt.s)))

In [281]:
mt = mt.filter_rows(mt.predicted_most_severe_consequence.contains('gain'))

In [282]:
mt = mt.annotate_rows(gts_non_ref = gts.index(mt.row_key).gts_non_ref,
                                gts_ref = gts.index(mt.row_key).gts_ref,
                                percent_gts_non_ref = gts.index(mt.row_key).percent_gts_non_ref,
                                percent_gts_ref = gts.index(mt.row_key).percent_gts_ref,
                                percent_gts_genotyped = gts.index(mt.row_key).percent_gts_genotyped)

In [283]:
mt = mt.filter_rows(mt.sports_non_ref > 1)

In [284]:
mt.count()

(2, 147)

In [285]:
to_export = mt.rows()
to_export = to_export.to_pandas()

In [286]:
to_export.columns

Index(['locus.contig', 'locus.position', 'alleles', 'rsid',
       'gnomAD_v3_1_allele_frequency', 'gnomAD_v3_1_homozygote_count',
       'predicted_most_severe_consequence', 'gene', 'hpo', 'controls_non_ref',
       'sports_non_ref', 'samples_het', 'samples_hom_non_ref', 'gts_non_ref',
       'gts_ref', 'percent_gts_non_ref', 'percent_gts_ref',
       'percent_gts_genotyped'],
      dtype='object')

In [287]:
to_export.to_csv('rare_variants_sportsmen')

In [288]:
to_export

,locus.contig,locus.position,alleles,rsid,gnomAD_v3_1_allele_frequency,gnomAD_v3_1_homozygote_count,predicted_most_severe_consequence,gene,hpo,controls_non_ref,sports_non_ref,samples_het,samples_hom_non_ref,gts_non_ref,gts_ref,percent_gts_non_ref,percent_gts_ref,percent_gts_genotyped
0,chr15,89315318,"[C, T]",rs121918164,0.000093,0,stop_gained,"[FANCI, POLG]","[[HP:0000072:Hydroureter, HP:0001053:Hypopigme...",0,3,"[B439, B480, B482]",[],0,40,0.0,100.0,100.0
1,chr16,5239977,"[G, T]",rs1472046975,0.000032,0,stop_gained,[RBFOX1],[None],2,3,"[B382, B431, B533, S_7213, S_7241]",[],1,38,2.5,95.0,97.5


### variant analysis in TTN (1 - filtering, 2 - fisher)

In [376]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [377]:
mt.count()

(3154875, 147)

In [378]:
mt = mt.filter_rows(mt.within_gene.contains('TTN'))

In [379]:
mt = mt.filter_rows((mt.sports_non_ref == 0), keep=False)

In [383]:
mt = mt.filter_rows(mt.gnomad_v3.v3_nfe.AF < 0.005)

In [384]:
mt.count()

(177, 147)

In [385]:
#select impactful variants

mt.aggregate_rows(hl.agg.counter(mt.gnomad_v3.most_severe_consequence))

{'missense_variant': 112,
 'splice_region_variant': 5,
 'inframe_deletion': 3,
 'synonymous_variant': 49,
 '3_prime_UTR_variant': 8}

In [386]:
to_test = mt.rows()
to_test = to_test.select(to_test.controls_non_ref, to_test.sports_non_ref, to_test.controls_ref, to_test.sports_ref)

to_test.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_ttn.ht')

2021-04-23 21:46:08 Hail: INFO: wrote table with 177 rows in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_ttn.ht
    Total size: 65.90 KiB
    * Rows: 65.89 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  92 rows (1.61 KiB)


In [387]:
to_test = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/to_test_ttn.ht')

In [388]:
to_test.count()

177

In [389]:
to_test = to_test.annotate(fisher = hl.fisher_exact_test(hl.int32(to_test.controls_non_ref), 
                                                         hl.int32(to_test.controls_ref),
                                                         hl.int32(to_test.sports_non_ref),
                                                         hl.int32(to_test.sports_ref)))

In [390]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sporstmen-005-intragenic-filtered-better.mt')

In [391]:
mt_test = mt.filter_rows(hl.is_defined(to_test[mt.row_key]), keep = True)

In [392]:
mt_test = mt_test.annotate_rows(fisher = to_test[mt_test.row_key]['fisher'])

In [393]:
mt_test = mt_test.select_rows(rsid = mt_test.gnomad_v3.rsid,
                                 gnomAD_v3_1_allele_frequency = mt_test.gnomad_v3.v3_nfe.AF,
                                 gnomAD_v3_1_homozygote_count = mt_test.gnomad_v3.v3_nfe.homozygote_count,
                                 predicted_most_severe_consequence = mt_test.gnomad_v3.most_severe_consequence,
                                 gene = mt_test.within_gene,
                                 hpo = mt_test.hpo,
                                 controls_non_ref = mt_test.controls_non_ref,
                                 sports_non_ref = mt_test.sports_non_ref,
                                 controls_ref = mt_test.controls_ref,
                                 sports_ref = mt_test.sports_ref,
                                 percent_controls_non_ref = mt_test.controls_non_ref/47*100,
                                 percent_sports_non_ref = mt_test.sports_non_ref/100*100,
                                 percent_controls_ref = mt_test.controls_ref/47*100,
                                 percent_sports_ref = mt_test.sports_ref/100*100,
                                 percent_sports_genotyped = (mt_test.sports_non_ref + mt_test.sports_ref)/100*100,
                                 percent_controls_genotyped = (mt_test.controls_non_ref + mt_test.controls_ref)/47*100,
                                 fisher = mt_test.fisher.p_value,
                                 )

In [394]:
mt_test = mt_test.filter_rows(hl.is_snp(mt_test.alleles[0], mt_test.alleles[1]))

In [395]:
mt_test = mt_test.annotate_rows(
    samples_het = hl.agg.filter(mt_test.GT.is_het(), hl.agg.collect(mt_test.s)),
    samples_hom_non_ref = hl.agg.filter(mt_test.GT.is_hom_var(), hl.agg.collect(mt_test.s)))

In [396]:
mt_test.count()

(173, 147)

In [397]:
### annotate with frequency in GTS:

gts = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/unrelated-gts.mt')

gts = gts.annotate_rows(gts_non_ref = hl.agg.count_where(gts.GT.is_non_ref()),
                        gts_ref = hl.agg.count_where(gts.GT.is_hom_ref()))

gts = gts.rows()

gts = gts.select(gts_non_ref = gts.gts_non_ref,
                 gts_ref = gts.gts_ref,
                 percent_gts_non_ref = gts.gts_non_ref/40*100,
                 percent_gts_ref = gts.gts_ref/40*100,
                 percent_gts_genotyped = (gts.gts_non_ref + gts.gts_ref)/40*100)

mt_test = mt_test.annotate_rows(gts_non_ref = gts.index(mt_test.row_key).gts_non_ref,
                                gts_ref = gts.index(mt_test.row_key).gts_ref,
                                percent_gts_non_ref = gts.index(mt_test.row_key).percent_gts_non_ref,
                                percent_gts_ref = gts.index(mt_test.row_key).percent_gts_ref,
                                percent_gts_genotyped = gts.index(mt_test.row_key).percent_gts_genotyped)

In [398]:
to_export = mt_test.rows()

to_export = to_export.to_pandas()

In [399]:
to_export.columns

Index(['locus.contig', 'locus.position', 'alleles', 'rsid',
       'gnomAD_v3_1_allele_frequency', 'gnomAD_v3_1_homozygote_count',
       'predicted_most_severe_consequence', 'gene', 'hpo', 'controls_non_ref',
       'sports_non_ref', 'controls_ref', 'sports_ref',
       'percent_controls_non_ref', 'percent_sports_non_ref',
       'percent_controls_ref', 'percent_sports_ref',
       'percent_sports_genotyped', 'percent_controls_genotyped', 'fisher',
       'samples_het', 'samples_hom_non_ref', 'gts_non_ref', 'gts_ref',
       'percent_gts_non_ref', 'percent_gts_ref', 'percent_gts_genotyped'],
      dtype='object')

In [400]:
to_export.shape

(173, 27)

In [401]:
to_export

,locus.contig,locus.position,alleles,rsid,gnomAD_v3_1_allele_frequency,gnomAD_v3_1_homozygote_count,predicted_most_severe_consequence,gene,hpo,controls_non_ref,...,percent_sports_genotyped,percent_controls_genotyped,fisher,samples_het,samples_hom_non_ref,gts_non_ref,gts_ref,percent_gts_non_ref,percent_gts_ref,percent_gts_genotyped
0,chr2,178525997,"[T, C]",rs72629798,0.003082,0,3_prime_UTR_variant,"[TTN, TTN-AS1]",[[HP:0003687:Centrally nucleated skeletal musc...,0,...,93.0,100.0,0.550771,"[B459, B81]",[],1,38,2.5,95.0,97.5
1,chr2,178526270,"[T, C]",rs72629797,0.000093,0,3_prime_UTR_variant,"[TTN, TTN-AS1]",[[HP:0003687:Centrally nucleated skeletal musc...,0,...,93.0,100.0,1.000000,[B457],[],0,39,0.0,97.5,97.5
2,chr2,178527006,"[G, T]",None,0.000774,0,3_prime_UTR_variant,"[TTN, TTN-AS1]",[[HP:0003687:Centrally nucleated skeletal musc...,1,...,93.0,100.0,1.000000,"[B518, WGS_37b]",[],0,40,0.0,100.0,100.0
3,chr2,178527288,"[T, C]",rs55832587,0.001657,0,synonymous_variant,"[TTN, TTN-AS1]",[[HP:0003687:Centrally nucleated skeletal musc...,0,...,93.0,100.0,0.550771,"[B435, B505]",[],0,39,0.0,97.5,97.5
4,chr2,178529113,"[C, T]",rs56324602,0.001642,0,synonymous_variant,"[TTN, TTN-AS1]",[[HP:0003687:Centrally nucleated skeletal musc...,0,...,93.0,100.0,0.550771,"[B435, B505]",[],0,40,0.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,chr2,178795088,"[C, G]",rs56128843,0.001843,0,missense_variant,[TTN],[[HP:0003687:Centrally nucleated skeletal musc...,0,...,93.0,100.0,0.550771,"[B435, B505]",[],0,39,0.0,97.5,97.5
169,chr2,178800552,"[G, A]",None,0.001750,0,synonymous_variant,[TTN],[[HP:0003687:Centrally nucleated skeletal musc...,0,...,93.0,100.0,0.550771,"[B435, B505]",[],0,39,0.0,97.5,97.5
170,chr2,178804577,"[G, A]",rs143623862,0.000062,0,synonymous_variant,[TTN],[[HP:0003687:Centrally nucleated skeletal musc...,0,...,93.0,100.0,1.000000,[B477],[],0,40,0.0,100.0,100.0
171,chr2,178830033,"[C, T]",rs76671760,0.001858,0,3_prime_UTR_variant,"[AC092640.1, CCDC141, TTN]","[[HP:0000771:Gynecomastia, HP:0000054:Micropen...",0,...,93.0,100.0,0.550771,"[B435, B505]",[],0,39,0.0,97.5,97.5


In [403]:
to_export.to_csv('ttn')